# 카프카(Kafka) 란 무엇인가?


### 카프카 이전
- 데이터를 전송하는 source 어플리케이션과 데이터를 받는 target 어플리케이션이 있었음
- 처음에는 간단한 단방향 커뮤니케이션을 수행했었음

![ex_screenshot](./kafka_img/screenshot1.png)

![ex_screenshot](./kafka_img/screenshot2.png)

- 문제: 
    - source/target 어플리케이션의 종류가 많아짐 -> 데이터 전송 라인이 매우 복잡
    - 데이터 전송 라인이 많아짐 -> 배포와 대응(respond)가 어려워짐
    - 데이터 전송시, 프로토콜/포맷의 파편화가 심각해짐
    - 추후 내부 데이터 포맷의 변경사항이 있을때 유지/보수 등이 매우 어려워짐

- Apache Kafka(아파치 카프카)는 이런 복잡함을 해결하기 위해 linkedin 에서 개발 및 오픈소스 제공


![ex_screenshot](./kafka_img/screenshot3.png)

- Apache Kafka
    - source 어플리케이션과 target 어플리케이션의 커플링을(내부 복잡도로 이해하면 될듯)
      약하게 하기위해 나옴
    - source 어플에서 카프카에 데이터 전송
    - target 어플은 카프카에서 데이터 가져옴

![ex_screenshot](./kafka_img/screenshot4.png)

- Features
    - source 어플(Kafka Producer):
        - 쇼핑몰의 클릭로그, 결제로그 등을 보냄
        - 보내는 데이터의 포맷은 제한이 없음(json, tsv, avro etc)
    - target 어플(Kafka Consumer):
        - 해당 로그를 적재, 처리 함
    - kafka 에는 데이터를 답는 토픽(topic)들이 존재함 (Queue라 생각하면 됨)

![ex_screenshot](./kafka_img/screenshot5.png)

- Kafka 특징
    - Fault Tolerant(고가용성): 서버에 문제가 생기거나, 랙(전원이) 내려간다던가<br> 하더라도 데이터 손실 없이 복구 가능
    - 낮은 지연(latency), 높은 처리량(Throughput)을 통해 대용량 데이터를 효과적으로 처리 가능<br> -> 빅데이터 처리에 매우 유용

- Kafka Topic?
    - 다양한 데이터가 들어갈 수 있는 공간
    - AMQP 와는 다르게 동작(?)
    - 데이터베이스의 Table이나 파일시스템의 폴더와 유사한 성질 가짐
    - 이름을 가질 수 있음 -> 목적에 따라 어떠한 데이터를 담는지 명확하게 명명
    
    

![ex_screenshot](./kafka_img/screenshot6.png)

- Topic
    - 하나의 토픽은 여러개의 Partition 으로 구성 (위에서 click_log, send_sms 등에 해당)
    - Partition은 큐와같이 데이터가 파티션 끝에서부터 순차적으로 쌓임 
    - Consumer가 파티션에 붙으면 가장 오래된 데이터부터 가져감
    - 더이상의 데이터가 없으면, Consumer는 다른 데이터가 들어올떄까지 대기함
    - **Consumer 가 데이터를 가져가더라도 데이터는 삭제되지 않음 **

![ex_screenshot](./kafka_img/screenshot7.png)

- Topic
    - 새로운 Consumer 가 붙게되면 동일한 데이터를(0번  부터) 가져갈 수 있음
    - 다만 Consumer 그룹이 다를경우 auto.offset.reset = earliest 라고 설정되어 있어야 함
    - 동일한 데이터에 대해 두번 이상 처리할 수 있는데 이것이 카프카를 사용하는 아주 중요한 이유라고 함
    - 클릭로그를 분석/시각하기 위해 Elastic Search에 저장하기도 하고, <br> 클릭로그를 백업하기위해 Hadoop에 저장하기도 함

![ex_screenshot](./kafka_img/screenshot8.png)

![ex_screenshot](./kafka_img/screenshot9.png)

- Topic에 Partition 이 2개 이상인 경우
    - Partition 이 새로 추가되었고 때마침 새로운 데이터가 들어오는 경우:
        - 데이터를 보낼때 키를 지정할 수 있음
        - 1) 키가 null 이고(키를 지정하지 않고), 기본 파티션너로 설정하면 round-robin(으로 파티션이 지정
        - 2) 키가 있고, 기본 파티션을 사용할경우 -> 키의 해시(hash)값을 구하고 해당 파티션에 할당

![ex_screenshot](./kafka_img/screenshot10.png)

* 키를 지정하지 않은경우, 파티션 0 번이 아닌 파티션 1번에 새로운 데이터(7)가 들어감
* 이후의 데이터 또한 round-robin으로 파티션 0 과 1에 나뉘어 들어감

![ex_screenshot](./kafka_img/screenshot11.png)
<br>
- 파티션을 늘릴때는 조심해야함 -> 파티션은 늘릴 수 있지만 줄일 수는 없음!
- 그럼에도 늘리는 이유?
    - Consumer 의 개수를 늘려 데이터 분산처리가 가능토록 함!

![ex_screenshot](./kafka_img/screenshot12.png)
<br>
- 그렇다면 파티션의 데이터(record)는 언제 삭제되나?
    - 옵션에 따라 다름
    - 데이터(record) 가 저장되는 최대 보존 시간과 최대 보존 크기를 직접 지정 가능
    - 일정한 기간/용량 동안 데이터를 저장 및 삭제 가능